# Reglas de asociación

In [16]:
import pandas as pd
import ETL as etl
import numpy as np
import os

from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
# Get query files function
def get_path(folder, file_path):
    current_directory = os.getcwd()
    return os.path.join(current_directory, '..', folder, file_path)

In [18]:
procesos_data = pd.read_csv(get_path('data', 'collected_obra_data_v3.csv'),
                 dtype = {'ID_ADJUDICACION': str},
                 parse_dates=['START_DATE','CONTRACT_DATE'])

In [20]:
def to_quantile(variable, data):
    #variable = 'NUM_PENALTIES_ENTIDAD_LAST_Y'

    percentile = [25,50,75]
    thresholds = np.percentile(data[variable], percentile)
    thresholds = [0] + list(thresholds) + [np.max(data[variable]) + 1]
    categories = np.digitize(data[variable], thresholds)

    etiquetas = ['Muy bajo','Bajo','Medio', 'Alto','Muy Alto']

    # Mapear los índices a las etiquetas personalizadas
    etiquetas_binarias = [etiquetas[i-1] for i in categories]

    data[variable + '_QUANTILE'] = etiquetas_binarias

    return data

variables = ['NUM_PENALTIES_ENTIDAD_LAST_Y', 'PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y', 'NUM_PENALTIES_ENTIDAD_ACTIVE','PROJECT_INTENSITY_NORM']

for var in variables:
    procesos_data_2 = to_quantile(var, procesos_data)

In [21]:
variables = ['NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE', 'PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE', 
             'NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE','PROJECT_INTENSITY_NORM_QUANTILE',
             'MUNICIPALITY_TYPE', 'COST_GROUP_QUANTILE', 'REGION', 'HAVE_DEVIATION']


procesos_data_2 = procesos_data_2[variables]

procesos_data_2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 8 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE          167 non-null    object
 1   PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE  167 non-null    object
 2   NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE          167 non-null    object
 3   PROJECT_INTENSITY_NORM_QUANTILE                167 non-null    object
 4   MUNICIPALITY_TYPE                              167 non-null    object
 5   COST_GROUP_QUANTILE                            167 non-null    object
 6   REGION                                         167 non-null    object
 7   HAVE_DEVIATION                                 167 non-null    bool  
dtypes: bool(1), object(7)
memory usage: 9.4+ KB


In [23]:
procesos_data_2 = pd.get_dummies(procesos_data_2, )
procesos_data_2

,HAVE_DEVIATION,NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto,PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto,PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Bajo,PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Medio,PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Muy bajo,NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto,PROJECT_INTENSITY_NORM_QUANTILE_Alto,PROJECT_INTENSITY_NORM_QUANTILE_Bajo,PROJECT_INTENSITY_NORM_QUANTILE_Medio,...,MUNICIPALITY_TYPE_TYPE_6,COST_GROUP_QUANTILE_Alto,COST_GROUP_QUANTILE_Bajo,COST_GROUP_QUANTILE_Medio,COST_GROUP_QUANTILE_Muy Alto,REGION_AMAZONIA,REGION_ANDINA,REGION_CARIBE,REGION_ORINOQUIA,REGION_PACIFICA
0,False,True,False,False,False,True,True,False,True,False,...,True,False,False,True,False,False,True,False,False,False
1,True,True,False,True,False,False,True,True,False,False,...,False,False,False,False,True,False,True,False,False,False
2,True,True,True,False,False,False,True,False,True,False,...,True,True,False,False,False,False,True,False,False,False
3,False,True,False,False,True,False,True,False,False,True,...,True,True,False,False,False,False,True,False,False,False
4,True,True,False,True,False,False,True,False,False,True,...,True,True,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,False,True,False,False,False,True,True,False,True,False,...,True,False,False,True,False,False,True,False,False,False
163,False,True,True,False,False,False,True,False,False,True,...,True,False,False,False,True,False,True,False,False,False
164,True,True,True,False,False,False,True,False,False,True,...,True,True,False,False,False,False,False,False,False,True
165,False,True,False,False,False,True,True,False,False,True,...,False,False,True,False,False,False,True,False,False,False


In [24]:
lista_soporte = apriori(procesos_data_2, min_support = 0.03, use_colnames = True)
lista_soporte

,support,itemsets
0,0.419162,(HAVE_DEVIATION)
1,1.000000,(NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto)
2,0.251497,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE...
3,0.245509,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE...
4,0.251497,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE...
...,...,...
1302,0.041916,"(COST_GROUP_QUANTILE_Medio, NUM_PENALTIES_ENTI..."
1303,0.035928,"(PROJECT_INTENSITY_NORM_QUANTILE_Muy bajo, COS..."
1304,0.035928,"(COST_GROUP_QUANTILE_Medio, PROJECT_INTENSITY_..."
1305,0.035928,"(MUNICIPALITY_TYPE_TYPE_6, NUM_PENALTIES_ENTID..."


In [25]:
# Calcular las métricas para las reglas de asociación y filtrar aquellas con mínimo lift
reglas = association_rules(lista_soporte, metric = "confidence", min_threshold = 0.5)
reglas

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(HAVE_DEVIATION),(NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto),0.419162,1.000000,0.419162,1.000000,1.000000,0.000000,inf,0.000000
1,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE...,(HAVE_DEVIATION),0.251497,0.419162,0.161677,0.642857,1.533673,0.056259,1.626347,0.464889
2,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE...,(HAVE_DEVIATION),0.245509,0.419162,0.125749,0.512195,1.221951,0.022841,1.190719,0.240741
3,(HAVE_DEVIATION),(NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto),0.419162,1.000000,0.419162,1.000000,1.000000,0.000000,inf,0.000000
4,(PROJECT_INTENSITY_NORM_QUANTILE_Alto),(HAVE_DEVIATION),0.251497,0.419162,0.125749,0.500000,1.192857,0.020331,1.161677,0.216000
...,...,...,...,...,...,...,...,...,...,...
6322,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE...,"(NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, P...",0.077844,0.203593,0.041916,0.538462,2.644796,0.026068,1.725549,0.674397
6323,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE...,(NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alt...,0.053892,0.221557,0.041916,0.777778,3.510511,0.029976,3.502994,0.755877
6324,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE...,(NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alt...,0.077844,0.203593,0.041916,0.538462,2.644796,0.026068,1.725549,0.674397
6325,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE...,"(COST_GROUP_QUANTILE_Bajo, NUM_PENALTIES_ENTID...",0.065868,0.179641,0.041916,0.636364,3.542424,0.030084,2.255988,0.768315


In [30]:
pd.set_option('display.max_colwidth', None)
# Filtrar reglas por ítem
reglas[(reglas['consequents'] == {'HAVE_DEVIATION'}) & (reglas['lift'] > 1)].sort_values(by='lift', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4822,"(NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, MUNICIPALITY_TYPE_TYPE_5, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto, COST_GROUP_QUANTILE_Muy Alto, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto)",(HAVE_DEVIATION),0.041916,0.419162,0.041916,1.000000,2.385714,0.024347,inf,0.606250
3093,"(COST_GROUP_QUANTILE_Muy Alto, MUNICIPALITY_TYPE_TYPE_5, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto)",(HAVE_DEVIATION),0.041916,0.419162,0.041916,1.000000,2.385714,0.024347,inf,0.606250
752,"(MUNICIPALITY_TYPE_TYPE_5, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto)",(HAVE_DEVIATION),0.041916,0.419162,0.041916,1.000000,2.385714,0.024347,inf,0.606250
1008,"(COST_GROUP_QUANTILE_Muy Alto, MUNICIPALITY_TYPE_TYPE_5, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto)",(HAVE_DEVIATION),0.041916,0.419162,0.041916,1.000000,2.385714,0.024347,inf,0.606250
2587,"(COST_GROUP_QUANTILE_Muy Alto, MUNICIPALITY_TYPE_TYPE_5, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto)",(HAVE_DEVIATION),0.041916,0.419162,0.041916,1.000000,2.385714,0.024347,inf,0.606250
135,"(MUNICIPALITY_TYPE_TYPE_5, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto)",(HAVE_DEVIATION),0.041916,0.419162,0.041916,1.000000,2.385714,0.024347,inf,0.606250
2508,"(MUNICIPALITY_TYPE_TYPE_5, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto)",(HAVE_DEVIATION),0.041916,0.419162,0.041916,1.000000,2.385714,0.024347,inf,0.606250
973,"(MUNICIPALITY_TYPE_TYPE_5, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto)",(HAVE_DEVIATION),0.041916,0.419162,0.041916,1.000000,2.385714,0.024347,inf,0.606250
3083,"(PROJECT_INTENSITY_NORM_QUANTILE_Alto, REGION_ANDINA, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto)",(HAVE_DEVIATION),0.053892,0.419162,0.047904,0.888889,2.120635,0.025315,5.227545,0.558544
1004,"(PROJECT_INTENSITY_NORM_QUANTILE_Alto, REGION_ANDINA, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto)",(HAVE_DEVIATION),0.053892,0.419162,0.047904,0.888889,2.120635,0.025315,5.227545,0.558544
